# WDI plots

Before ingestion, using data downloaded from https://datatopics.worldbank.org/world-development-indicators/ as CSV. 
Currently, all of the plots are created using the same plot function just to get an overview of trends. The plot type, labels or interpolation might not be suitable for all indicators. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Butchering to keep only rows with country level and world data (no continents, income groups, etc.)
df = pd.read_csv('WDICSV.csv', skiprows=range(1, 71810))

# identify columns that represent years
year_columns = df.columns[4:]  

# transform df to long-form
df_long = pd.melt(df, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], 
                  value_vars=year_columns, 
                  var_name='Year', 
                  value_name='Value')

# convert 'Year' to numeric
df_long['Year'] = pd.to_numeric(df_long['Year'])

# remove erroneous data entry for Tonga in 2002
df_long.loc[(df_long['Country Code'] == 'TON') & 
            (df_long['Year'] == 2002) & 
            (df_long['Indicator Code'] == 'SE.SEC.ENRR.MA'), 'Value'] = np.nan

df_long.loc[(df_long['Country Code'] == 'TON') & 
            (df_long['Year'] == 2002) & 
            (df_long['Indicator Code'] == 'SE.SEC.ENRR'), 'Value'] = np.nan

# back to wide format for easier plotting
df_wide = df_long.pivot_table(index=['Country Name', 'Country Code', 'Year'], 
                              columns='Indicator Code',
                              values='Value').reset_index()

# Display to verify (we should have 1498 columns)
print(df_wide.head())

### Plot function

In [ ]:
def plot_indicator(df_wide, df_long, indicator_code, selected_countries, start_year=1980, interpolate=True):
    
    """
    
    Plot a specified indicator over time for all countrie, with selected countries and the world average highlighted.

    The function plots the trend of a specified indicator from a given starting year onward.
    All available countries are shown in light grey, with selected countries highlighted in distinct colors.
    A dashed black line represents the world value for that indicator.

    When including interpolation, missing data points within each country's series are handled with log-linear interpolation 
    for gaps up to 5 years, within the range of existing data.

    Parameters:
    ----------
    df_wide : pd.DataFrame
        The wide-format DataFrame containing yearly data for multiple indicators. 
        Must contain columns for 'Country Name', 'Country Code', 'Year', and the specified indicator.
    
    df_long : pd.DataFrame
        The long-format DataFrame containing yearly data for multiple indicators, with separate
        columns for 'Indicator Code' and 'Indicator Name'. This is used to retrieve the indicator's name.
    
    indicator_code : str
        The code of the WDI indicator(s) to plot.
    
    selected_countries : list of str
        List of country names to be highlighted in the plot. These countries will be shown 
        in distinct colors with labels, while other countries are displayed in light grey.
    
    start_year : int, optional (default=1980)
        The starting year for the plot. Data before this year will be excluded from the plot.

    interpolate (bool): 
        Whether to apply log-linear interpolation for missing data. Default=True.

    Behavior:
    ----------
    - Interpolates missing values within each country's series using log-linear interpolation, 
      filling gaps up to 5 years where data is missing. Only data points within the range 
      of existing data (from the first to last valid entry) are interpolated.
    - Uses the world value for the indicator directly from the dataset (rows where 'Country Code' is 'WLD').
    - The plot displays each country’s data with light grey lines, highlights selected countries 
      with labeled lines, and includes a dashed black line for the world.

    Example Usage:
    ----------
        selected_countries = ['Afghanistan', 'United States', 'China', 'India']
        plot_indicator(df_wide, df_long, 'SE.SEC.ENRR', selected_countries, start_year=1990, interpolate=False)
    // 
        selected_countries = ['United States', 'Ethiopia', 'China', 'Iraq']
        plot_indicator(df_wide, df_long, 'NY.GDP.PCAP.KD', selected_countries, start_year=1980, interpolate=True)

    """

    # Extract indicator name from the long-format df
    indicator_name = df_long[df_long['Indicator Code'] == indicator_code]['Indicator Name'].iloc[0]
    
    # Filter for years from start_year onwards and exclude 'WLD' country code
    df_filtered = df_wide[df_wide['Year'] >= start_year][['Country Name', 'Country Code', 'Year', indicator_code]]

    # Interpolation
    if interpolate:
        df_filtered[indicator_code] = df_filtered[indicator_code].where(df_filtered[indicator_code] > 0, np.nan)
        df_filtered[indicator_code] = np.log(df_filtered[indicator_code])

        # Interpolate in log scale for each country
        df_filtered[indicator_code] = df_filtered.groupby('Country Code', group_keys=False)[indicator_code].apply(
            lambda group: group.interpolate(method='linear', limit_area='inside')
        )
        # back to original scale
        df_filtered[indicator_code] = np.exp(df_filtered[indicator_code])

    # Separate world data
    world_data = df_filtered[df_filtered['Country Code'] == 'WLD']

    # Plot all other countries in grey
    plt.figure(figsize=(12, 7))
    for country in df_filtered['Country Name'].unique():
        country_data = df_filtered[df_filtered['Country Name'] == country]
        plt.plot(
            country_data['Year'], 
            country_data[indicator_code], 
            color='lightgrey', 
            linewidth=0.6,    
            alpha=0.9      
        )

    # Highlight selected countries
    for country in selected_countries:
        country_data = df_filtered[df_filtered['Country Name'] == country]
        plt.plot(country_data['Year'], country_data[indicator_code], label=country, linewidth=2.5)
    
    # Plot the world
    plt.plot(world_data['Year'], world_data[indicator_code], label='World', color='black', linestyle='--', linewidth=2.5)
    years=df_filtered['Year'][df_filtered[indicator_code].notnull()]
    tick_gap = np.round((years.max() - years.min()) / (5)).astype(int)
    ticks = [round(years.min() + i * tick_gap) for i in range(6)]
    plt.xticks(ticks, fontsize=12)
    plt.yticks(fontsize=12)
    #plt.xticks(ticks)
    
    # Customize
    plt.ylabel(indicator_name, fontsize=12) 
    plt.title(f'{indicator_name} Over Time by Country', fontsize=12)

    plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12, title_fontsize=14)
    plt.grid(True)
    plt.tight_layout()  
    plt.show()    

In [ ]:
# Define selected countries
selected_countries = ['Afghanistan', 'United States', 'China', 'India', 'Chile', 'Ethiopia', 'Indonesia', 'Nigeria', 'Russia', 'Saudi Arabia']

# List of indicator codes to plot
indicator_codes = [
'SN.ITK.DEFC.ZS',
'SH.H2O.SMDW.ZS',
'EG.ELC.ACCS.ZS',
'EG.ELC.ACCS.UR.ZS',
'SL.GDP.PCAP.EM.KD',
'EG.EGY.PRIM.PP.KD',
'MS.MIL.XPND.ZS',  
'SL.TLF.TOTL.FE.ZS', 
'SM.POP.TOTL.ZS', 
'SP.DYN.IMRT.IN', 
'SH.DYN.MORT.FE',
'SP.POP.0014.FE.ZS', 
'SP.POP.1564.FE.ZS', 
'SP.POP.65UP.FE.ZS', 
'SP.URB.TOTL.IN.ZS',
'SM.POP.REFG.OR', 
'SP.DYN.IMRT.FE.IN', 
'NY.GDP.PCAP.KD', 
'SP.DYN.LE00.IN', 
'SE.PRM.NENR']

# Loop through each indicator code and plot
for indicator_code in indicator_codes:
    plot_indicator(df_wide, df_long, indicator_code, selected_countries)

### other viewser indicators

In [ ]:
# Define the selected countries
selected_countries = ['Afghanistan', 'United States', 'China', 'India', 'Chile', 'Ethiopia', 'Indonesia', 'Nigeria', 'Russia', 'Saudi Arabia']

# List of indicator codes to plot
indicator_codes = [
 'SP.POP.65UP.MA.ZS', 
 'SP.POP.DPND', 
 'SE.SEC.NENR', 
 'SH.H2O.BASW.ZS',  
 'SP.DYN.TFRT.IN', 
 'SL.UEM.ADVN.MA.ZS', 
 'SP.POP.1564.MA.ZS', 
 'SP.POP.0014.MA.ZS']

# Loop through each indicator code and plot
for indicator_code in indicator_codes:
    plot_indicator(df_wide, df_long, indicator_code, selected_countries)

##### Log-linear interpolation not suitable 

In [ ]:
# Define the selected countries
selected_countries = ['Afghanistan', 'United States', 'China', 'India', 'Chile', 'Ethiopia', 'Indonesia', 'Nigeria', 'Russia', 'Saudi Arabia']

# Indicator codes to plot
indicator_codes_zeroes = [
'SE.SEC.ENRR', 
'SE.SEC.CUAT.LO.ZS', 
'SE.SEC.ENRR.FE', 
'SE.SEC.ENRR.MA', 
'SH.STA.SMSS.ZS', 
'EG.CFT.ACCS.ZS', 
'ER.MRN.PTMR.ZS', 
'VC.IHR.PSRC.P5', 
'DT.ODA.ODAT.GN.ZS', 
'EN.ATM.PM25.MC.T2.ZS', 
'SE.ENR.PRIM.FM.ZS', 
'SE.ENR.PRSC.FM.ZS', 
'SH.STA.STNT.ZS', 
'SH.STA.MALN.ZS', 
'SM.POP.NETM', 
'SP.POP.GROW', 
'NY.ADJ.DFOR.CD', 
'EN.GHG.CO2.MT.CE.AR5', 
'EN.GHG.CO2.IP.MT.CE.AR5', 
'AG.LND.FRST.K2', 
'ER.FSH.CAPT.MT', # threatened fish species only available for 1 single year, switching to tonnes of fish captured (but then also less meaningful for SDG14)
'SP.M15.2024.FE.ZS'] #(world data not available, and too much missing data to calculate a meaningful world average)

for indicator_code in indicator_codes_zeroes:
    plot_indicator(df_wide, df_long, indicator_code, selected_countries, interpolate=False)

### Lots of missing data/no data for world:

In [ ]:
df_filtered = pd.read_csv('WDICSV.csv', skiprows=range(1, 73305)) # skipping world rows here to avoid misleading lines in plots with absolute values, remove if needed    

# Display first few rows to verify (should start with Afghanistan and EG.CFT.ACCS.ZS)
print(df_filtered.head())

# Identify year columns
year_columns = df_filtered.columns[4:]  

# Transform df to long-form
df_long = pd.melt(df_filtered, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], 
                  value_vars=year_columns, 
                  var_name='Year', 
                  value_name='Value')

# Convert 'Year' to numeric
df_long['Year'] = pd.to_numeric(df_long['Year'])

# back to wide format for easier plotting
df_wide_filtered = df_long.pivot_table(index=['Country Name', 'Country Code', 'Year'], 
                              columns='Indicator Code',
                              values='Value').reset_index()

# Plot function with world average
def plot_indicator_2(df_wide_filtered, df_long, indicator_code, selected_countries, start_year=1980):
    """
    Same as above, apart from world value. Here, World value is calculated as the average of all available countries.
    """
    # Extract indicator name 
    indicator_name = df_long[df_long['Indicator Code'] == indicator_code]['Indicator Name'].iloc[0]
    
    # Filter for years from start_year onwards and select columns
    df_filtered = df_wide_filtered[df_wide_filtered['Year'] >= start_year][['Country Name', 'Country Code', 'Year', indicator_code]]

    # log-linear interpolation
    df_filtered[indicator_code] = df_filtered[indicator_code].where(df_filtered[indicator_code] > 0, np.nan)  # Set <= 0 to NaN
    df_filtered[indicator_code] = np.log(df_filtered[indicator_code])  

    # Interpolate in the log scale for each country
    df_filtered[indicator_code] = df_filtered.groupby('Country Code', group_keys=False)[indicator_code].apply(
        lambda group: group.interpolate(method='linear', limit=5, limit_area='inside')
    )

    # back to original scale
    df_filtered[indicator_code] = np.exp(df_filtered[indicator_code])

    # Calculate average value per year across all countries
    world_avg = df_filtered.groupby('Year')[indicator_code].mean().reset_index()

    # Plot all other countries in grey 
    plt.figure(figsize=(12, 7))
    for country in df_filtered['Country Name'].unique():
        country_data = df_filtered[df_filtered['Country Name'] == country]
        plt.plot(
            country_data['Year'], 
            country_data[indicator_code], 
            color='lightgrey', 
            linewidth=0.6,    
            alpha=0.9      
        )

    # Highlight selected countries 
    for country in selected_countries:
        country_data = df_filtered[df_filtered['Country Name'] == country]
        plt.plot(country_data['Year'], country_data[indicator_code], label=country, linewidth=2.5)

    # Plot the world average (mean value of available data, not nessecarily mean of all countries or actual world average)
    plt.plot(world_avg['Year'], world_avg[indicator_code], label='World Average', color='black', linestyle='--', linewidth=2.5)
    
    # Customize plot
    years = df_filtered['Year'][df_filtered[indicator_code].notnull()]
    tick_gap = np.round((years.max() - years.min()) / (5)).astype(int)
    ticks = [round(years.min() + i * tick_gap) for i in range(6)]
    plt.xticks(ticks, fontsize=12)
    plt.yticks(fontsize=12)

    plt.xlabel('Year')
    plt.ylabel(indicator_name) 
    plt.title(f'{indicator_name} Over Time by Country')

    plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12, title_fontsize=14)
    plt.grid(True)
    plt.tight_layout()  
    plt.show()

In [ ]:
indicator_codes = [
'NY.GDP.MKTP.PP.KD',
'NY.GDP.MKTP.CD',
'NY.GNP.MKTP.PP.KD', 
'IQ.CPA.ENVR.XQ', # lots of missing data
'IQ.CPA.PUBS.XQ', # lots of missing data
'SG.DMK.SRCR.FN.ZS', # world data not available, and too much missing data to calculate a meaningful world average
'EN.GHG.ALL.MT.CE.AR5',
'SP.POP.TOTL', 
'NV.AGR.TOTL.CN', 
'NV.AGR.TOTL.KD',
'NV.AGR.TOTL.KN',  
'AG.LND.TOTL.RU.K2',
'AG.LND.TOTL.K2',
'AG.SRF.TOTL.K2', 
'SL.UEM.ADVN.FE.ZS', 
'SL.UEM.NEET.ZS', 
'SL.UEM.NEET.FE.ZS', 
'SL.UEM.ADVN.ZS',
'NV.MNF.TECH.ZS.UN' 
] 

# Loop through each indicator code and plot
for indicator_code in indicator_codes:
    plot_indicator_2(df_wide_filtered, df_long, indicator_code, selected_countries)

#### Table of indicator codes used and corresponding names 

In [ ]:
used_indicator_codes = [
'SN.ITK.DEFC.ZS', 
'SE.SEC.ENRR',
'SE.SEC.CUAT.LO.ZS',
'SE.SEC.ENRR.FE',
'SE.SEC.ENRR.MA',
'SH.H2O.SMDW.ZS',
'SH.STA.SMSS.ZS',
'EG.ELC.ACCS.ZS',
'EG.ELC.ACCS.UR.ZS',
'EG.CFT.ACCS.ZS',
'SL.GDP.PCAP.EM.KD',
'SG.DMK.SRCR.FN.ZS',
'ER.MRN.PTMR.ZS',
'VC.IHR.PSRC.P5',
'DT.ODA.ODAT.GN.ZS',
'EG.EGY.PRIM.PP.KD', 
'EN.ATM.PM25.MC.T2.ZS',
'MS.MIL.XPND.GD.ZS', 
'MS.MIL.XPND.ZS', 
'SE.ENR.PRIM.FM.ZS', 
'SE.ENR.PRSC.FM.ZS', 
'SH.STA.STNT.ZS', 
'SH.STA.MALN.ZS',
'SL.TLF.TOTL.FE.ZS', 
'SM.POP.NETM', 
'SM.POP.TOTL.ZS', 
'SP.DYN.IMRT.IN', 
'SH.DYN.MORT.FE',
'SP.POP.0014.FE.ZS', 
'SP.POP.1564.FE.ZS', 
'SP.POP.65UP.FE.ZS', 
'SP.POP.GROW',
'SP.URB.TOTL.IN.ZS',
'SM.POP.REFG.OR', 
'SP.DYN.IMRT.FE.IN', 
'NY.GDP.PCAP.KD', 
'SP.DYN.LE00.IN', 
'SE.PRM.NENR', 
'NY.ADJ.DFOR.CD', 
'SP.POP.65UP.MA.ZS', 
'SP.POP.DPND', 
'SE.SEC.NENR', 
'SH.H2O.BASW.ZS',  
'SP.DYN.TFRT.IN', 
'SL.UEM.ADVN.MA.ZS', 
'SP.POP.1564.MA.ZS', 
'SL.UEM.ADVN.FE.ZS', 
'SL.UEM.NEET.ZS', 
'SL.UEM.NEET.FE.ZS', 
'SP.POP.0014.MA.ZS', 
'SL.UEM.ADVN.ZS',
'NY.GDP.MKTP.PP.KD',
'NY.GDP.MKTP.CD',
'NY.GNP.MKTP.PP.KD', 
'EN.GHG.ALL.MT.CE.AR5',
'EN.GHG.CO2.MT.CE.AR5',
'EN.GHG.CO2.IP.MT.CE.AR5',
'IQ.CPA.ENVR.XQ',
'IQ.CPA.PUBS.XQ',
'SP.POP.TOTL', 
'NV.AGR.TOTL.CN', 
'NV.AGR.TOTL.KD',
'NV.AGR.TOTL.KN',  
'AG.LND.FRST.K2', 
'AG.LND.TOTL.RU.K2',
'AG.LND.TOTL.K2',
'AG.SRF.TOTL.K2', 
'ER.FSH.CAPT.MT',
'NV.MNF.TECH.ZS.UN',
'SP.M15.2024.FE.ZS'] 

# Filter for indicator table
indicator_table = df_long[df_long['Indicator Code'].isin(used_indicator_codes)][['Indicator Code', 'Indicator Name']].drop_duplicates()

# Reset index for readability
indicator_table = indicator_table.reset_index(drop=True)

# Display
pd.set_option('display.max_rows', None) 
print(indicator_table)
display(indicator_table)